In [ ]:
#Web Scraping y filtro para Minciencias y separacion de productos

In [ ]:
#pip install openpyxl

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from scrapy.selector import Selector
import requests
from bs4 import BeautifulSoup
import warnings

In [2]:
warnings.filterwarnings('ignore')
warnings.warn('InsecureRequestWarning')
warnings.warn('Do not show this message')

In [3]:
dfgrupos = pd.read_excel("BDMicro.xlsx", engine='openpyxl') #, sheet_name='Sheet1'
tamaño_dfgrupos =dfgrupos.shape
url= dfgrupos["LINKS"]
dfcon = pd.DataFrame()

In [4]:
url

0     https://scienti.minciencias.gov.co/gruplac/jsp...
1     https://scienti.minciencias.gov.co/gruplac/jsp...
2     https://scienti.minciencias.gov.co/gruplac/jsp...
3     https://scienti.minciencias.gov.co/gruplac/jsp...
4     https://scienti.minciencias.gov.co/gruplac/jsp...
5     https://scienti.minciencias.gov.co/gruplac/jsp...
6     https://scienti.minciencias.gov.co/gruplac/jsp...
7     https://scienti.minciencias.gov.co/gruplac/jsp...
8     https://scienti.minciencias.gov.co/gruplac/jsp...
9     https://scienti.minciencias.gov.co/gruplac/jsp...
10    https://scienti.minciencias.gov.co/gruplac/jsp...
11    https://scienti.minciencias.gov.co/gruplac/jsp...
12    https://scienti.minciencias.gov.co/gruplac/jsp...
13    https://scienti.minciencias.gov.co/gruplac/jsp...
14    https://scienti.minciencias.gov.co/gruplac/jsp...
15    https://scienti.minciencias.gov.co/gruplac/jsp...
16    https://scienti.minciencias.gov.co/gruplac/jsp...
17    https://scienti.minciencias.gov.co/gruplac

In [6]:
#Para el ejemplo 
#url = url[0:5]

In [5]:
palabras_filtro = "agua"
palabras_filtro2 = "microbiolog"


####OJO EL CICLO### 
for i in range (0,tamaño_dfgrupos [0]): #tamaño_dfgrupos [0]
    count = i
    try:
        print("Procesando link  ",count," .")
        url = dfgrupos["LINKS"][i]
        r = requests.get(url, verify=False)
        soup = BeautifulSoup(r.text, 'lxml')
        NomGrupo = soup.find("span", class_='celdaEncabezado')
        NombreGrupo = NomGrupo.text
        #Filtrar todas las tablas
        table = soup.find_all('table')
        df = pd.read_html(str(table))
        #Numero de de tablas 
        NumTablas = len(df)
        #CICLO 
        dfcon1 = pd.DataFrame()
        dfcon1["Index"] = [i]
        dfcon1["Nombre del Grupo"] = [NombreGrupo]
        #Tabla numero 0, datos basicos, que es diferente 
        #Tabla de datos 
        df = pd.read_html(str(table))[0]
        #Imputacion de los NAN
        df= df.fillna('Sin dato')
        #Tabla 1 
        tamaño = df.shape
        Nombre = df[0][0]
        Dato = []
        for j in range(1,tamaño[0]):
            Dato.append(df[0][j]+":  "+df[1][j]+"... ")
        Datos = " \n ".join(Dato)
        Datos
        dfcon1[Nombre]= [Datos]
        # Siguientes Tablas 1-5
        print("Procesando link  ",count," ..")
        for i in range (1,6):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]):
                Datos.append(df[0][j])
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos] 
        print("Procesando link  ",count," ...")
        # Siguientes Tablas 6-12
        for i in range (6,13):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            df_filtrado = df[df[0].str.contains(palabras_filtro,case=False) & df[0].str.contains(palabras_filtro2,case=False) ] 
            Datos = []
            for j in range(0,tamaño[0]):
                try:
                    Datos.append(df_filtrado[0][j])
                except:
                    pass
            Datos = " \n ".join(Datos) 
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," ....")
        #Tabla 13 -44
        for i in range (13,45):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]):
                try:
                    df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]               
                    Datos.append(df_filtrado[1][j])
                except:
                    pass
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," .....")
        #Table 45 - 46
        for i in range (45,47):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]-1):
                df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]
                Datos.append(df[1][j])
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos]
        print("Procesando link  ",count," ......")
        #Tabla 47 - 84 
        for i in range (47,84):
            df = pd.read_html(str(table))[i]
            tamaño = df.shape
            Nombre = df[0][0]
            Datos = []
            for j in range(1,tamaño[0]-1):
                try:
                    df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False)] #Case = false para que no importen mayusculas
                    Datos.append(df_filtrado[1][j])
                except:
                    pass
            Datos = " \n ".join(Datos)
            dfcon1[Nombre]= [Datos] 
        dfcon = pd.concat([dfcon, dfcon1])
    except:
            print ("error en link  ",count)
dfcon = dfcon.set_index("Index")
dfcon

Procesando link   0  .
Procesando link   0  ..
Procesando link   0  ...
Procesando link   0  ....
Procesando link   0  .....
Procesando link   0  ......
Procesando link   1  .
Procesando link   1  ..
Procesando link   1  ...
Procesando link   1  ....
Procesando link   1  .....
Procesando link   1  ......
Procesando link   2  .
Procesando link   2  ..
Procesando link   2  ...
Procesando link   2  ....
Procesando link   2  .....
Procesando link   2  ......
Procesando link   3  .
Procesando link   3  ..
Procesando link   3  ...
Procesando link   3  ....
Procesando link   3  .....
error en link   3
Procesando link   4  .
Procesando link   4  ..
Procesando link   4  ...
Procesando link   4  ....
Procesando link   4  .....
Procesando link   4  ......
Procesando link   5  .
Procesando link   5  ..
Procesando link   5  ...
Procesando link   5  ....
Procesando link   5  .....
Procesando link   5  ......
Procesando link   6  .
Procesando link   6  ..
Procesando link   6  ...
Procesando link   6 

,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,Participación Ciudadana en Proyectos de CTI,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos
Index,,,,,,,,,,,,,,,,,,,,,
0,"Microbiología, Industria y Ambiente GIMIA",Año y mes de formación: 2013 - 1... \n Depar...,1.- UNIVERSIDAD SANTIAGO DE CALI - (Avalado),Plan de trabajo: Reuniones con periodicidad Me...,1.- Microbiología Ambiental \n 2.- Microbiolog...,Nombre \n 1.- Iván Andrés GONZÁLEZ VARGAS \n 2...,,,,,...,,,,,,13.- Trabajos de grado de pregrado : Revision ...,,15.- Pregrado : Revisión Sistemática: Bacteria...,,
1,PARASITOLOGIA - AGROECOLOGIA MILENIO,Año y mes de formación: 2002 - 2... \n Depar...,1.- UNIVERSIDAD DEL MAGDALENA - (No Avalado) \...,Plan de trabajo: El grupo de investigación Par...,1.- Bioprospección \n 2.- Biotecnología Agrope...,Nombre \n 1.- Pedro Jose Fragoso Castilla \n 2...,,,,,...,,,,,,3.- Trabajos de grado de pregrado : EVALUACIÓN...,,24.- Curso de perfeccionamiento/especializació...,,
2,GRESBIOCA,Año y mes de formación: 2014 - 1... \n Depar...,1.- UNIVERSIDAD POPULAR DEL CESAR - (Avalado),Plan de trabajo: *Consolidar el grupo de inves...,1.- Recursos Naturales Y Ecología \n 2.- Pedag...,Nombre \n 1.- Jean Carlos Ospino Jimenez \n 2....,,,,,...,,,,,,,,,,
4,Grupo de Biotecnología Ambiental e Industrial,Año y mes de formación: 1996 - 6... \n Depar...,1.- PONTIFICIA UNIVERSIDAD JAVERIANA - (Avalado),Plan de trabajo: El grupo de Biotecnología Amb...,1.- Sistemas biológicos no convencionales y pr...,Nombre \n 1.- Raúl Alberto Poutou Piñales \n 2...,,,,,...,,,,,,31.- Trabajos de grado de pregrado : Tratamien...,,26.- Pregrado : Evaluación de un tratamiento b...,,17.- Demás trabajos : Evaluación de la Calidad...
5,Grupo de Investigación en Química y Biotecnolo...,Año y mes de formación: 2002 - 1... \n Depar...,1.- UNIVERSIDAD SANTIAGO DE CALI - (Avalado),Plan de trabajo: Con base en la experiencia de...,1.- Desarrollos Tecnológicos y Biotecnologicos...,Nombre \n 1.- Richard Fernando Dvries Arturo \...,,,,,...,,,,,,,,60.- Pregrado : Validación secundaria del NMP ...,,
6,Ecología y agricultura inteligente para la sos...,Año y mes de formación: 1995 - 3... \n Depar...,1.- Corporación Colombiana De Investigación Ag...,Plan de trabajo: La gestión sostenible de los ...,1.- Agricultura de precisión \n 2.- Agroclimat...,Nombre \n 1.- César Augusto Terán Chaves \n 2....,,,,,...,,,,,,,,,,
7,Unipluriversidad,Año y mes de formación: 1998 - 3... \n Depar...,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: Siguiendo con el cumplimiento...,1.- Educación Superior y Desarrollo Regional \...,Nombre \n 1.- Hader de Jesús Calderón Serna \n...,,,,,...,,,,,,,,,,
8,BIOALI Biotecnología de Alimentos,Año y mes de formación: 2004 - 8... \n Depar...,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: Estado del arte: Objetivos: A...,1.- Desarrollo de Envases activos e inteligent...,Nombre \n 1.- Oscar Alfonso Vega Castro \n 2.-...,,,,,...,,,,,,,,,,
9,Grupo de Investigaciones Mellitopalinológicas ...,Año y mes de formación: 2000 - 2... \n Depar...,1.- UNIVERSIDAD DEL TOLIMA - (Avalado),Plan de trabajo: El Grupo de Investigaciones M...,1.- Actividad biológica de alimentos \n 2.- Es...,Nombre \n 1.- Guillermo Salamanca Grosso \n 2....,,,,,...,,,,,,,,,,


In [6]:
dfcon.to_excel("InormacionFiltrada.xlsx")

In [7]:
df = pd.read_excel("InormacionFiltrada.xlsx",engine='openpyxl') #sheet_name='Sheet1', 
df = df.set_index("Index")

In [8]:
#leer el DataFrame e imprimir cada columna como una tabla nueva

In [9]:
##Eliminar los sin dato y los vacios (:  -)

#Crear tabla de datos basicos
DatosBasicos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
for j in range(0,df.shape[0]):
    try:
        dfDatosBasidos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
        ListaDatosBasicos = df["Datos básicos"][j].split("... ")
        for i in range (0,len(ListaDatosBasicos)-1):
            if i == 0:
                dfDatosBasidos.loc[len(dfDatosBasidos)] = [df["Nombre del Grupo"][j], ListaDatosBasicos[i]]
            else:
                dfDatosBasidos.loc[len(dfDatosBasidos)] = ["", ListaDatosBasicos[i]]
        #REMOVER LAS FILAS QUE NO SIRVEN 
        v1 = dfDatosBasidos.iloc[:, 1].str.contains("sin dato",case=False)
        v2 = dfDatosBasidos.iloc[:, 1].str.contains(":  -",case=False)
        for i in range (0,len(ListaDatosBasicos)-1):
            if (v1[i] == True) or (v2[i] == True):
                dfDatosBasidos = dfDatosBasidos.drop([i],axis=0)
        DatosBasicos = pd.concat([DatosBasicos, dfDatosBasidos])
    except: 
        pass
DatosBasicos.to_excel("DatosBasicos.xlsx")
DatosBasicos

,Nombre del Grupo,Datos básicos
0,"Microbiología, Industria y Ambiente GIMIA",Año y mes de formación: 2013 - 1
1,,\n Departamento - Ciudad: VALLE DEL CAUCA - ...
2,,\n Líder: Iván Andrés GONZÁLEZ VARGAS
3,,\n ¿La información de este grupo se ha certif...
5,,\n E-mail: investigacionfcb@usc.edu.co
...,...,...
5,,\n E-mail: gbiotecnologiaccm@gmail.com
6,,\n Clasificación: C con vigencia hasta la p...
7,,\n Área de conocimiento: Ciencias Agrícolas ...
8,,\n Programa nacional de ciencia y tecnología:...


In [10]:
# Crear tabla de informacion de los grupos 
DatosGrupo = df.iloc[:,0:6].drop(["Datos básicos"],axis=1)
DatosGrupo.to_excel("DatosGrupo.xlsx")
DatosGrupo

,Nombre del Grupo,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo
Index,,,,,
0,"Microbiología, Industria y Ambiente GIMIA",1.- UNIVERSIDAD SANTIAGO DE CALI - (Avalado),Plan de trabajo: Reuniones con periodicidad Me...,1.- Microbiología Ambiental \n 2.- Microbiolog...,Nombre \n 1.- Iván Andrés GONZÁLEZ VARGAS \n 2...
1,PARASITOLOGIA - AGROECOLOGIA MILENIO,1.- UNIVERSIDAD DEL MAGDALENA - (No Avalado) \...,Plan de trabajo: El grupo de investigación Par...,1.- Bioprospección \n 2.- Biotecnología Agrope...,Nombre \n 1.- Pedro Jose Fragoso Castilla \n 2...
2,GRESBIOCA,1.- UNIVERSIDAD POPULAR DEL CESAR - (Avalado),Plan de trabajo: *Consolidar el grupo de inves...,1.- Recursos Naturales Y Ecología \n 2.- Pedag...,Nombre \n 1.- Jean Carlos Ospino Jimenez \n 2....
4,Grupo de Biotecnología Ambiental e Industrial,1.- PONTIFICIA UNIVERSIDAD JAVERIANA - (Avalado),Plan de trabajo: El grupo de Biotecnología Amb...,1.- Sistemas biológicos no convencionales y pr...,Nombre \n 1.- Raúl Alberto Poutou Piñales \n 2...
5,Grupo de Investigación en Química y Biotecnolo...,1.- UNIVERSIDAD SANTIAGO DE CALI - (Avalado),Plan de trabajo: Con base en la experiencia de...,1.- Desarrollos Tecnológicos y Biotecnologicos...,Nombre \n 1.- Richard Fernando Dvries Arturo \...
6,Ecología y agricultura inteligente para la sos...,1.- Corporación Colombiana De Investigación Ag...,Plan de trabajo: La gestión sostenible de los ...,1.- Agricultura de precisión \n 2.- Agroclimat...,Nombre \n 1.- César Augusto Terán Chaves \n 2....
7,Unipluriversidad,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: Siguiendo con el cumplimiento...,1.- Educación Superior y Desarrollo Regional \...,Nombre \n 1.- Hader de Jesús Calderón Serna \n...
8,BIOALI Biotecnología de Alimentos,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: Estado del arte: Objetivos: A...,1.- Desarrollo de Envases activos e inteligent...,Nombre \n 1.- Oscar Alfonso Vega Castro \n 2.-...
9,Grupo de Investigaciones Mellitopalinológicas ...,1.- UNIVERSIDAD DEL TOLIMA - (Avalado),Plan de trabajo: El Grupo de Investigaciones M...,1.- Actividad biológica de alimentos \n 2.- Es...,Nombre \n 1.- Guillermo Salamanca Grosso \n 2....


In [11]:
# Crear todas las tablas en archivos diferentes. 

df1 = df.set_index("Nombre del Grupo")
for i in range(5,84):
    try:
        NombreColumna = df1.columns[i]
        Nombre = str(i)+' '+NombreColumna+".xlsx"
        dfsin_nan = df1[NombreColumna].dropna()
        try:
            if type(dfsin_nan[0]) != str():
                dfsin_nan.to_excel(Nombre)
        except:
            pass
    except:
        pass